In [3]:
import pandas as pd
import preprocessing
import numpy as np

d:\EID-Download\02 Academic\Python38\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")

#coach data
coach_time = pd.read_excel("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\caoching_time.xlsx")


Preprocessing Fitbit data

In [5]:
merged_final = preprocessing.merge_data(dailyactivity, wear_time, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min)
select_survey = preprocessing.select_survey_data(survey_data)

d:\EIC-Code\00-Python\2024-08-Objective-identification-on-mental-health_CNN_LSTM\preprocessing.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
d:\EIC-Code\00-Python\2024-08-Objective-identification-on-mental-health_CNN_LSTM\preprocessing.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
d:\EIC-Code\00-Python\2024-08-Objective-identification-on-mental-health_CNN_LSTM\preprocessing.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_date

In [ ]:
fitbit_data = merged_final.copy()
#replace code 98 as 107 because the code of this participant is 107 in survey
fitbit_data.loc[fitbit_data['Id'] == 98, 'Id'] = 107

used_var = ['Id','date','TotalMinutesWearTime', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories', 'RestingHeartRate','TotalActiveZoneMinutes', 'FatBurnActiveZoneMinutes',
       'CardioActiveZoneMinutes', 'PeakActiveZoneMinutes', 'DailyRMSSD', 'DeepRMSSD',
       'AvgBreathsPerMinute', 'LightSleepAvgBreathsPerMinute','DeepSleepAvgBreathsPerMinute', 'REMSleepAvgBreathsPerMinute',
       'TotalMinutesAsleep_x', 'TotalTimeInBed_x', 'TotalTimeAwake', 'TotalMinutesLight', 
       'TotalMinutesDeep','TotalMinutesREM', 'NightlyRelative', 'Value'] #delete 'Above', 'Below', Custom Zone; NightlyRelative represents ski temparature

used_fitbit_data = fitbit_data[used_var]

used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep','TotalTimeInBed_x':'TotalMinutesinbed',
                                   'TotalTimeAwake': 'TotalMinutesAwake',
                                   'Value': 'HR',
                                   'NightlyRelative': 'skin_temparature'}, 
                                   inplace = True)

#select participants matched with survey
all_survey_id = select_survey['Id'].unique()
used_fitbit_data = used_fitbit_data[used_fitbit_data['Id'].isin(all_survey_id)]
filtered_data = used_fitbit_data.groupby('Id').filter(lambda x: len(x) >= 10) #select dat which data points is larger than 10

In [ ]:
# impute time sequence data
miss_imputation = preprocessing.impute_missing(filtered_data)

#keep only one date
impute_data_frame = miss_imputation.copy()
impute_data_frame = impute_data_frame.reset_index().drop(columns = 'index')
date = impute_data_frame.iloc[:, 0]
impute_data_frame = impute_data_frame.drop(columns=['date'])
impute_data_frame['date'] = date

#Expand the time range in order to make the data to be time-series at the daily level because some of them missing some days data
impute_data_frame.insert(loc = 2, column = 'survey_date', value = np.nan) #add another column Fitbit data
new_data_frame = preprocessing.extend_time(impute_data_frame, select_survey)

#recode survey data
new_used_fitbit_data = preprocessing.recode_survey_time(new_data_frame, select_survey) #recode survey time, complete first follow-up, complete second follow-up

#add coach time
fitbit_coach = preprocessing.recode_coach_date(new_used_fitbit_data, coach_time) #recode coach date and combine with Fitbit

# impute extend data
miss_imputation2 = preprocessing.impute_missing(fitbit_coach)
#keep only one date
impute_data_frame2 = miss_imputation2.copy()
impute_data_frame2 = impute_data_frame2.reset_index().drop(columns = 'index')
date = impute_data_frame2.iloc[:, 0]
impute_data_frame2 = impute_data_frame2.drop(columns=['date'])
impute_data_frame2['date'] = date

impute_data_frame2.to_csv('impute_data_frame2.csv')

Data augumentation

In [21]:
impute_data_frame2 = pd.read_csv('impute_data_frame2.csv')
impute_data_frame2.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'], inplace = True)

In [22]:
def add_gaussian_noise(time_series, mean=0.0, stddev=1.0):
    """
    Adds Gaussian noise to a time series.
     Options:
     time_series (array-like): A time series to which noise is added.
     mean (float): The average value of the noise. Default is 0.0.
     stddev (float): Standard deviation of noise. Default is 1.0.

     Returns:
     noisy_series (np.array): Time series with added noise.
     """
     # Gaussian noise generation
    noise = np.random.normal(mean, stddev, len(time_series))

    # Adding noise to the original time series
    noisy_series = time_series + noise

    return noisy_series

In [23]:
data_augumentation = impute_data_frame2.copy()
non_need_augumentation = ['survey_date', 'coach_date', 'date']
data_augumentation.drop(columns = non_need_augumentation, inplace = True)

total_ids = data_augumentation['Id'].unique()
after_augumentation = pd.DataFrame()

for id in total_ids:
    individual_data = data_augumentation[data_augumentation['Id'] == id]
    individual_data = individual_data.drop(columns = 'Id').copy()

    augmented_data = individual_data.apply(lambda col: add_gaussian_noise(col, mean=0.0, stddev=0.05))
    augmented_data['Id'] = id
    after_augumentation = pd.concat([after_augumentation, augmented_data])

after_augumentation[['survey_date', 'coach_date', 'date']] = impute_data_frame2[['survey_date', 'coach_date', 'date']]
after_augumentation.set_index('Id', inplace = True)
after_augumentation.reset_index(inplace = True)

Preprocessing Survey data

In [ ]:
select_survey['weeks_followup_survey_96ac_complete'].value_counts() #87 sample size, We only select the participants who completed the second follow-up
#check the individual who did not complete second follow-up survey
select_survey[select_survey['weeks_followup_survey_96ac_complete'] == 0]['Id']
# 40      42
# 46      48
# 67      69
# 107    109dd
# 112    114
# 123    125
# 129    131
# 132    134

In [6]:
#handle outcome variables

select_survey_columns = ['nervous_v1', 'down_v1',  'blue_v1', 'nervous_v2', 'down_v2', 'blue_v2'] 
select_survey_columns2 = ['calm_v1','calm_v2','happy_v1','happy_v2']
#nervous: 24 (0, 20, 40, 60, 80, 100)
#down: 25 (0, 20, 40, 60, 80, 100)
#calm: 26 (100, 80, 60, 40, 20, 0)
#downhearted: 28 (0, 20, 40, 60, 80, 100) 
#happy: 30 (100, 80, 60, 40, 20, 0)

select_survey[select_survey_columns] = select_survey[select_survey_columns].replace({1:0, 2:20, 3:40, 4:60, 5:80, 6:100})
select_survey[select_survey_columns2] = select_survey[select_survey_columns2].replace({1:100, 2:80, 3:60, 4:40, 5:20, 6:0})

#category mental health, three levels
# The average of the five responses were used for data analysis
select_survey['mental_health_1'] = select_survey[['nervous_v1', 'down_v1', 'calm_v1','blue_v1', 'happy_v1']].mean(axis = 1)
select_survey['mental_health_2'] = select_survey[['nervous_v2', 'down_v2', 'calm_v2','blue_v2', 'happy_v2']].mean(axis = 1)


In [7]:
select_survey.to_csv('select_survey.csv')

In [24]:
#recode demongraphic variable
all_fitbit_id = data_augumentation['Id'].unique()

var = ['demographics_race___1','demographics_race___2','demographics_race___3','demographics_race___4',
       'demographics_race___5','demographics_race___6','demographics_race___7']
recode_demographic_variable = select_survey[var]
recode_demographic_variable['demographic'] = np.nan
recode_demographic_variable['Id'] = select_survey['Id']

#1: demographics_race___1 = 1
#2: demographics_race___2 = 1
#3: demographics_race___3 = 3
#4: demographics_race___4 = 4
#5: demographics_race___5 = 5
#6: demographics_race___6 = 6
#7: demographics_race___7 = 7
#8: multiple_race = 8

# Recoding logic
for i in all_fitbit_id:
    row = recode_demographic_variable[recode_demographic_variable['Id'] == i][var]
    # Check if there are multiple races selected
    if row.sum(axis=1).iloc[0] > 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 8  # Multiple races
    elif row['demographics_race___1'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 1
    elif row['demographics_race___2'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 2
    elif row['demographics_race___3'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 3
    elif row['demographics_race___4'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 4
    elif row['demographics_race___5'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 5
    elif row['demographics_race___6'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 6
    elif row['demographics_race___7'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 7

select_survey['demographics_race'] =  recode_demographic_variable['demographic']

used_variables = ['Id', 'current_status','demographics_age','demographics_sex','demographics_sexorient',
                  'demographics_ethnicity', 'demographics_immigration','demographics_race',
                  'demographics_education', 'demographics_sorority']

select_data = select_survey[used_variables]

C:\Users\karri\AppData\Local\Temp\ipykernel_20256\877023948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recode_demographic_variable['demographic'] = np.nan
C:\Users\karri\AppData\Local\Temp\ipykernel_20256\877023948.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recode_demographic_variable['Id'] = select_survey['Id']


Combine Fitbit data with survey data

In [25]:

#merge Fitbit data with survey data
fitbit_survey = pd.merge(after_augumentation, select_data, on = 'Id', how = 'left')

#move the date time
fitbit_survey.set_index('date', inplace=True)
fitbit_survey.reset_index(inplace=True)

In [26]:
# impute time sequence data
miss_imputation3 = preprocessing.impute_missing(fitbit_survey)
#keep only one date
impute_data_frame3 = miss_imputation3.copy()
impute_data_frame3 = impute_data_frame3.reset_index().drop(columns = 'index')
date = impute_data_frame3.iloc[:, 0]
impute_data_frame3 = impute_data_frame3.drop(columns=['date'])
impute_data_frame3['date'] = date

d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning origina

In [27]:
# academic day: weekday except break
# non-academic day: weekend, fallbreak, winterbreak, spring semester, spring break, summer holiday)

#weekend:
#----10.07 saturaday - non-academic day
#----10.08 sunday - non-academic day

#fall break "2023-10-8 :2023-10-15"
#winter break '2023-11-15:'2023-01-08'
#martine luda:  2024 January 15
#president date:  2024 February 19
#spring break: 2024 March 3-10
#summer break: '2023-5-1: 2023-8-21'

fall_break = pd.date_range(start='2023-10-8', end='2023-10-15')
winter_break = pd.date_range(start='2023-11-15', end='2024-01-08')
spring_break = pd.date_range(start='2024-03-03', end='2024-03-10')
summer_break = pd.date_range(start='2024-05-02', end='2024-05-13')
summer_break2 = pd.date_range(start='2024-08-02', end='2023-08-21')
other_days = ['2024-01-15', '2024-02-19']
other_days = pd.DatetimeIndex(pd.to_datetime(other_days))

# earlist_date = impute_data_frame3['date'].min()
# latest_date = impute_data_frame3['date'].max()

# # Generate a date range between the start and end dates
# determine_weekend = pd.date_range(start=earlist_date, end=latest_date)

# date_info = pd.DataFrame({
#     'date': determine_weekend,
#     'is_weekend': determine_weekend.weekday.isin([5, 6])  # True for Saturday (5) and Sunday (6)
# })

# # Filter to display only weekend days
# weekends = date_info[date_info['is_weekend']]['date'].values
# weekends = pd.DatetimeIndex(pd.to_datetime(weekends))

#combine all the non-academic day
non_academic_days = fall_break.union(winter_break).union(other_days).union(spring_break).union(summer_break).union(summer_break2)

#recode the date data
impute_data_frame3['date_code'] = impute_data_frame3['date'].apply(lambda x:0 if x in non_academic_days else 1)

fitbit_survey2 = impute_data_frame3.drop(columns = 'date')

In [ ]:
fitbit_survey2.to_csv("fitbit_survey2.csv")

,Id,TotalMinutesWearTime,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,RestingHeartRate,TotalActiveZoneMinutes,...,current_status,demographics_age,demographics_sex,demographics_sexorient,demographics_ethnicity,demographics_immigration,demographics_race,demographics_education,demographics_sorority,date_code
0,5.0,1440.057164,18236.041340,70.958623,24.929687,225.004400,538.973991,2746.981724,66.052424,174.968832,...,3.0,21.0,2.0,9.0,2.0,1.0,1.0,5.0,0.0,0
1,5.0,1214.050203,14563.024690,47.034864,5.978522,195.037633,821.004152,2394.054230,68.993880,111.911485,...,3.0,21.0,2.0,9.0,2.0,1.0,1.0,5.0,0.0,0
2,5.0,1399.929508,14769.997580,38.023865,23.009524,293.957780,379.001081,2571.997988,71.931790,85.098837,...,3.0,21.0,2.0,9.0,2.0,1.0,1.0,5.0,0.0,0
3,5.0,876.020191,6609.019169,33.050072,0.988976,90.972874,1155.980510,1929.977122,74.065840,71.936134,...,3.0,21.0,2.0,9.0,2.0,1.0,1.0,5.0,0.0,0
4,5.0,1428.989593,2867.033531,0.971538,6.907825,153.941051,713.115384,1814.022582,71.927814,26.946675,...,3.0,21.0,2.0,9.0,2.0,1.0,1.0,5.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5546,167.0,1427.417391,10463.342936,20.716445,35.331551,259.476266,645.701144,2440.574796,65.956320,49.338247,...,4.0,22.0,2.0,9.0,2.0,1.0,8.0,5.0,0.0,1
5547,167.0,1427.447119,10463.276280,20.819992,35.473158,259.475530,645.708350,2440.697140,65.910566,49.310811,...,4.0,22.0,2.0,9.0,2.0,1.0,8.0,5.0,0.0,1
5548,167.0,1427.414524,10463.318722,20.783083,35.409434,259.596501,645.750349,2440.555235,65.911357,49.233066,...,4.0,22.0,2.0,9.0,2.0,1.0,8.0,5.0,0.0,1
5549,167.0,1427.428079,10463.385049,20.855530,35.409579,259.596462,645.834127,2440.549602,65.922107,49.299660,...,4.0,22.0,2.0,9.0,2.0,1.0,8.0,5.0,0.0,1
